In [2]:
# Cài đặt thư viện
!pip install -q langchain langchain-community langchain-core neo4j transformers accelerate bitsandbytes torch pydantic

In [2]:
# Import libraries
import json
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.graphs import Neo4jGraph
from tqdm import tqdm
import time
from pydantic import BaseModel, Field
from typing import List

/home/duo/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Setup Neo4j connection
NEO4J_URI = "neo4j+s://0c367113.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gTO1K567hBLzkRdUAhhEb-UqvBjz0i3ckV3M9v_-Nio"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [3]:
chunk_file = "/home/duo/work/DL/btl/data/chunk/chunks_by_clause.jsonl"

# Load JSONL file (mỗi dòng là một JSON object)
chunks = []
with open(chunk_file, 'r', encoding='utf-8') as f:
    for line in f:
        chunk_data = json.loads(line.strip())
        chunks.append(chunk_data)

print(f"Đã load {len(chunks)} chunks từ file luật giao thông")
print(f"Ví dụ chunk đầu tiên: {chunks[0]['id']}")
print(f"Nội dung: {chunks[0]['content'][:200]}...")

Đã load 3770 chunks từ file luật giao thông
Ví dụ chunk đầu tiên: 36_2024_QH15_art1
Nội dung: Điều 1. Phạm vi điều chỉnh Luật này quy định về quy tắc, phương tiện, người tham gia giao thông đường bộ, chỉ huy, điều khiển, tuần tra, kiểm soát, giải quyết tai nạn giao thông đường bộ, trách nhiệm ...


In [ ]:
# Load Qwen2.5 model từ Hugging Face
print("Đang load model Qwen2.5-7B-Instruct...")

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True
)


In [ ]:
# Define Pydantic schemas cho validation
class Entity(BaseModel):
    """Schema cho một entity"""
    name: str = Field(description="Tên thực thể")
    type: str = Field(description="Loại thực thể: VEHICLE (phương tiện), REGULATION (quy định), PENALTY (mức phạt), ROAD_ELEMENT (yếu tố đường), VIOLATION (vi phạm), ORGANIZATION (cơ quan)")
    description: str = Field(description="Mô tả về thực thể")

class Relationship(BaseModel):
    """Schema cho một relationship"""
    source: str = Field(description="Tên entity nguồn")
    target: str = Field(description="Tên entity đích")
    type: str = Field(description="Loại quan hệ")
    description: str = Field(description="Mô tả quan hệ")

class KnowledgeGraph(BaseModel):
    """Schema cho toàn bộ knowledge graph output"""
    entities: List[Entity] = Field(default_factory=list, description="Danh sách entities")
    relationships: List[Relationship] = Field(default_factory=list, description="Danh sách relationships")

print("Pydantic schemas đã sẵn sàng!")
print(f"\nSchema: {KnowledgeGraph.model_json_schema()}")

In [ ]:
# Prompt template để extract entities và relationships
EXTRACTION_PROMPT = """Phân tích văn bản luật giao thông đường bộ Việt Nam sau và trích xuất entities và relationships.

QUY TẮC:
- Chỉ trích xuất thông tin CÓ TRONG văn bản
- Entity types: 
  + VEHICLE: phương tiện giao thông (xe máy, ô tô, xe thô sơ...)
  + PENALTY: mức phạt, hình thức xử lý
  + VIOLATION: hành vi vi phạm (vượt đèn đỏ, chạy quá tốc độ...)
  + ROAD_ELEMENT: phần đường, làn đường, vỉa hè, biển báo...
  + ORGANIZATION: cơ quan quản lý (Cảnh sát giao thông, Bộ GTVT...)
- Relationship bao gồm types sau:
  + BỊ_PHẠT: mối quan hệ giữa hành vi vi phạm và mức phạt
  + KHÔNG_BỊ_PHẠT: mối quan hệ giữa hành vi và mức phạt không áp dụng
  + ĐƯỢC_PHÉP: mối quan hệ giữa phương tiện và hành vi được phép thực hiện
  + KHÔNG_ĐƯỢC_PHÉP: mối quan hệ giữa phương tiện và hành vi không được phép
  + CẤM: mối quan hệ giữa phương tiện và hành vi bị cấm
  Và các loại quan hệ khác nếu cần thiết, đảm bảo mô tả chính xác mối quan hệ giữa các entity.

VĂN BẢN:
{text}

Hãy trả về kết quả theo ĐÚNG định dạng JSON sau (ĐẢM BẢO cú pháp JSON hợp lệ):
{{
  "entities": [
    {{"name": "Xe máy", "type": "VEHICLE", "description": "Phương tiện giao thông cơ giới đường bộ"}},
    {{"name": "Vượt đèn đỏ", "type": "VIOLATION", "description": "Hành vi vi phạm tín hiệu đèn giao thông"}},
    {{"name": "Phạt tiền từ 4-6 triệu đồng", "type": "PENALTY", "description": "Mức phạt vi phạm đèn tín hiệu"}}
  ],
  "relationships": [
    {{"source": "Xe máy", "target": "Vượt đèn đỏ", "type": "BỊ_PHẠT", "description": "Xe máy bị phạt khi vượt đèn đỏ"}},
    {{"source": "Vượt đèn đỏ", "target": "Phạt tiền từ 4-6 triệu đồng", "type": "BỊ_PHẠT", "description": "Vi phạm đèn đỏ bị phạt tiền"}}
  ]
}}

CHỈ TRẢ VỀ JSON, KHÔNG GIẢI THÍCH THÊM:"""

def generate_structured(prompt: str) -> str:
    """Generate text với Qwen model"""
    messages = [
        {"role": "system", "content": "Bạn là một chuyên gia phân tích văn bản luật giao thông đường bộ Việt Nam. Trả về kết quả dưới dạng JSON hợp lệ."},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2046,  # Giảm để nhanh hơn
            temperature=0.1,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
def extract_knowledge(text: str, max_retries: int = 3) -> dict:
    """Extract entities và relationships từ text - validate với Pydantic, có retry khi lỗi"""
    
    for attempt in range(max_retries):
        try:
            # Tạo prompt
            prompt = EXTRACTION_PROMPT.format(text=text)
            
            # Generate response
            response = generate_structured(prompt)
            
            # Parse JSON từ response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                data = json.loads(json_str)
                
                # Validate với Pydantic
                kg = KnowledgeGraph(**data)
                
                # Convert to dict
                result = {
                    "entities": [e.model_dump() for e in kg.entities],
                    "relationships": [r.model_dump() for r in kg.relationships]
                }
                
                # Thành công
                if attempt > 0:
                    print(f"Retry thành công sau {attempt + 1} lần thử")
                return result
            else:
                print(f"Không tìm thấy JSON (lần {attempt + 1}/{max_retries})")
                if attempt == max_retries - 1:
                    return {"entities": [], "relationships": []}
                continue
            
        except json.JSONDecodeError as e:
            print(f"JSON decode error (lần {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                return {"entities": [], "relationships": []}
            continue
            
        except Exception as e:
            print(f"Error (lần {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                return {"entities": [], "relationships": []}
            continue
    
    return {"entities": [], "relationships": []}

In [ ]:
# Hàm thêm entities và relationships vào Neo4j (tối ưu cho luật giao thông)
def add_to_graph(knowledge: dict, chunk_metadata: dict):
    """Thêm knowledge vào Neo4j graph với metadata luật giao thông"""
    
    # Extract metadata
    law_code = chunk_metadata.get('law_code', 'UNKNOWN')
    law_name = chunk_metadata.get('law_name', '')
    article = chunk_metadata.get('article', 0)
    chapter = chunk_metadata.get('chapter', '')
    chapter_title = chunk_metadata.get('chapter_title', '')
    mode = chunk_metadata.get('mode', '')
    has_penalty = chunk_metadata.get('has_penalty', False)
    clauses = chunk_metadata.get('clauses', [])  # Lấy clauses thay vì vehicles
    
    # 1. Thêm entities KHÔNG CÓ metadata - chỉ name, type, description
    for entity in knowledge.get('entities', []):
        name = entity.get('name', '').strip()
        entity_type = entity.get('type', 'UNKNOWN')
        description = entity.get('description', '')
        
        if not name or len(name) < 2:  # Tránh entities quá ngắn
            continue
        
        # Query đơn giản - chỉ MERGE theo name và type, không lưu metadata
        query = f"""
        MERGE (e:{entity_type} {{name: $name}})
        ON CREATE SET 
            e.description = $description,
            e.created_at = datetime()
        ON MATCH SET
            e.description = CASE 
                WHEN e.description IS NULL OR e.description = '' 
                THEN $description 
                ELSE e.description 
            END,
            e.last_updated = datetime()
        RETURN e.name as name
        """
        
        try:
            graph.query(query, {
                'name': name,
                'description': description
            })
        except Exception as e:
            print(f"Error adding entity {name}: {e}")
    
    # 2. Thêm relationships VỚI METADATA ĐẦY ĐỦ
    for rel in knowledge.get('relationships', []):
        source = rel.get('source', '').strip()
        target = rel.get('target', '').strip()
        rel_type = rel.get('type', 'RELATED_TO').replace(' ', '_').replace('-', '_').upper()
        description = rel.get('description', '')
        
        if not source or not target or len(source) < 2 or len(target) < 2:
            continue
        
        # Query với MERGE relationship và lưu METADATA ĐẦY ĐỦ
        query = f"""
        MATCH (s {{name: $source}})
        MATCH (t {{name: $target}})
        MERGE (s)-[r:{rel_type}]->(t)
        ON CREATE SET 
            r.description = $description,
            r.mode = $mode,
            r.law_name = $law_name,
            r.law_code = $law_code,
            r.chapter = $chapter,
            r.chapter_title = $chapter_title,
            r.article = $article,
            r.clauses = $clauses,
            r.has_penalty = $has_penalty,
            r.created_at = datetime(),
            r.occurrence_count = 1
        ON MATCH SET
            r.last_seen_article = $article,
            r.occurrence_count = COALESCE(r.occurrence_count, 0) + 1,
            r.last_updated = datetime()
        RETURN type(r) as rel_type
        """
        
        try:
            graph.query(query, {
                'source': source,
                'target': target,
                'description': description,
                'mode': mode,
                'law_name': law_name,
                'law_code': law_code,
                'chapter': chapter,
                'chapter_title': chapter_title,
                'article': article,
                'clauses': clauses,
                'has_penalty': has_penalty
            })
        except Exception as e:
            # Nếu không tìm thấy entity, bỏ qua (có thể entity bị filter)
            pass

In [ ]:
# Xóa dữ liệu cũ
clear_old_data = False  # Đổi thành True nếu muốn xóa

if clear_old_data:
    graph.query("MATCH (n) DETACH DELETE n")
    print("Đã xóa dữ liệu cũ")
else:
    print("Giữ nguyên dữ liệu cũ")

In [ ]:
# Tạo indexes để tối ưu hiệu suất query
print("Đang tạo indexes cho Neo4j...")

indexes = [
    "CREATE INDEX vehicle_name IF NOT EXISTS FOR (v:VEHICLE) ON (v.name)",
    "CREATE INDEX violation_name IF NOT EXISTS FOR (v:VIOLATION) ON (v.name)",
    "CREATE INDEX penalty_name IF NOT EXISTS FOR (p:PENALTY) ON (p.name)",
    "CREATE INDEX regulation_name IF NOT EXISTS FOR (r:REGULATION) ON (r.name)",
    "CREATE INDEX road_element_name IF NOT EXISTS FOR (r:ROAD_ELEMENT) ON (r.name)",
    "CREATE INDEX organization_name IF NOT EXISTS FOR (o:ORGANIZATION) ON (o.name)",
]

for idx_query in indexes:
    try:
        graph.query(idx_query)
        print(f"Đã tạo index")
    except Exception as e:
        print(f"Index có thể đã tồn tại: {str(e)[:50]}")

print("\nIndexes đã sẵn sàng!")

In [ ]:
# BƯỚC 1: Extract knowledge và LƯU VÀO FILE JSON
import os

# Tạo thư mục output nếu chưa có
os.makedirs('/data/graph_output', exist_ok=True)

# File để lưu kết quả extraction
output_file = '/data/graph_output/extracted_knowledge.jsonl'

# Process chunks và lưu kết quả vào file
start_chunk = 0  
end_chunk = 30  
total_entities = 0
total_relationships = 0
errors = 0

num_chunks_to_process = min(end_chunk, len(chunks)) - start_chunk
print(f"Bắt đầu xử lý chunks từ {start_chunk} đến {end_chunk-1} (tổng {num_chunks_to_process})...")
print(f"Kết quả sẽ được lưu vào: {output_file}")

# Mở file để ghi (mode 'w' để ghi đè, 'a' để append)
with open(output_file, 'w', encoding='utf-8') as f:
    for i in tqdm(range(start_chunk, min(end_chunk, len(chunks))), desc="Extracting knowledge"):
        chunk = chunks[i]
        
        try:
            # Extract knowledge
            knowledge = extract_knowledge(chunk['content'])
            
            # Tạo record để lưu
            record = {
                'chunk_id': chunk.get('id', i),
                'chunk_index': i,
                'metadata': chunk['metadata'],
                'knowledge': knowledge
            }
            
            # Ghi vào file JSONL (mỗi dòng là 1 JSON object)
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            
            # Stats
            entities_count = len(knowledge.get('entities', []))
            rels_count = len(knowledge.get('relationships', []))
            total_entities += entities_count
            total_relationships += rels_count
            
            # Print progress
            if ((i - start_chunk + 1) % 5) == 0:
                print(f"\nProgress after {i-start_chunk+1} chunks:")
                print(f"   Entities extracted: {total_entities}")
                print(f"   Relationships extracted: {total_relationships}")
                print(f"   Errors: {errors}")
            
        except Exception as e:
            errors += 1
            print(f"\nError at chunk {i}: {e}")
            # Lưu error record
            error_record = {
                'chunk_id': chunk.get('id', i),
                'chunk_index': i,
                'error': str(e),
                'metadata': chunk['metadata']
            }
            f.write(json.dumps(error_record, ensure_ascii=False) + '\n')
            continue

print(f"\n{'='*60}")
print(f"HOÀN TẤT EXTRACTION!")
print(f"{'='*60}")
print(f"Thống kê:")
print(f"   - Chunks xử lý: {num_chunks_to_process}")
print(f"   - Tổng entities: {total_entities}")
print(f"   - Tổng relationships: {total_relationships}")
print(f"   - Errors: {errors}")
print(f"   - Success rate: {((num_chunks_to_process - errors) / num_chunks_to_process * 100):.1f}%")
print(f"\nKết quả đã được lưu vào: {output_file}")
print(f"Chạy cell tiếp theo để upload lên Neo4j!")


In [ ]:
# BƯỚC 2: Load từ file JSON và UPLOAD LÊN NEO4J (với batch processing)

# File chứa kết quả extraction
input_file = '/home/duo/work/DL/btl/data/graph_output/extracted_knowledge.jsonl'

# Batch size để giảm số lượng request (quan trọng cho Neo4j Aura Free)
BATCH_SIZE = 10  # Có thể điều chỉnh tùy theo giới hạn

print(f"Đang load dữ liệu từ: {input_file}")

# Load tất cả records
records = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line.strip())
        # Chỉ lấy records có knowledge (bỏ qua error records)
        if 'knowledge' in record:
            records.append(record)

print(f"Đã load {len(records)} records hợp lệ")

# Upload lên Neo4j theo batch
total_uploaded = 0
total_entities_uploaded = 0
total_relationships_uploaded = 0
upload_errors = 0

print(f"\nBắt đầu upload lên Neo4j (batch size: {BATCH_SIZE})...")

for batch_start in tqdm(range(0, len(records), BATCH_SIZE), desc="Uploading batches"):
    batch_end = min(batch_start + BATCH_SIZE, len(records))
    batch = records[batch_start:batch_end]
    
    # Process từng record trong batch
    for record in batch:
        try:
            knowledge = record['knowledge']
            metadata = record['metadata']
            
            # Add to graph
            add_to_graph(knowledge, metadata)
            
            total_uploaded += 1
            total_entities_uploaded += len(knowledge.get('entities', []))
            total_relationships_uploaded += len(knowledge.get('relationships', []))
            
        except Exception as e:
            upload_errors += 1
            print(f"\nError uploading record {record.get('chunk_index', '?')}: {e}")
            continue
    
    # Small delay giữa các batch để tránh rate limit
    if batch_end < len(records):
        time.sleep(1)
    
    # Progress report mỗi 5 batches
    if ((batch_start // BATCH_SIZE + 1) % 5) == 0:
        print(f"\nProgress: {total_uploaded}/{len(records)} records uploaded")
        print(f"   Entities: {total_entities_uploaded}")
        print(f"   Relationships: {total_relationships_uploaded}")
        print(f"   Errors: {upload_errors}")

print(f"\n{'='*60}")
print(f"HOÀN TẤT UPLOAD!")
print(f"{'='*60}")
print(f"Thống kê:")
print(f"   - Records uploaded: {total_uploaded}/{len(records)}")
print(f"   - Tổng entities: {total_entities_uploaded}")
print(f"   - Tổng relationships: {total_relationships_uploaded}")
print(f"   - Upload errors: {upload_errors}")
print(f"   - Success rate: {(total_uploaded / len(records) * 100):.1f}%")


In [ ]:
# Kiểm tra kết quả trong Neo4j (tối ưu cho luật giao thông)
print("=== Thống kê Entities theo loại ===")
count_query = """
MATCH (n)
RETURN labels(n)[0] as Type, count(n) as Count
ORDER BY Count DESC
"""
results = graph.query(count_query)
total_entities = 0
for row in results:
    print(f"  {row['Type']}: {row['Count']}")
    total_entities += row['Count']
print(f"\nTổng: {total_entities} entities")

print("\n=== Thống kê Relationships ===")
rel_count_query = """
MATCH ()-[r]->()
RETURN type(r) as RelType, count(r) as Count
ORDER BY Count DESC
LIMIT 20
"""
rel_results = graph.query(rel_count_query)
total_rels = 0
for row in rel_results:
    print(f"  {row['RelType']}: {row['Count']}")
    total_rels += row['Count']
print(f"\nTổng relationships (top 20): {total_rels}")

print("\n=== Thống kê theo Luật (từ relationships) ===")
law_query = """
MATCH ()-[r]->()
WHERE r.law_code IS NOT NULL
RETURN r.law_code as LawCode, r.law_name as LawName, count(r) as Count
ORDER BY Count DESC
LIMIT 10
"""
law_results = graph.query(law_query)
for row in law_results:
    law_name = row.get('LawName', '')[:50]
    print(f"  {row['LawCode']}: {row['Count']} relationships - {law_name}")

print("\n=== Thống kê Entities có mức phạt ===")
penalty_query = """
MATCH (p:PENALTY)
RETURN count(p) as PenaltyCount
"""
penalty_results = graph.query(penalty_query)
for row in penalty_results:
    print(f"  Tổng mức phạt: {row['PenaltyCount']}")

print("\n=== Thống kê Vi phạm ===")
violation_query = """
MATCH (v:VIOLATION)
OPTIONAL MATCH (v)-[r:BỊ_PHẠT]->(p:PENALTY)
RETURN count(v) as ViolationCount, count(r) as WithPenalty
"""
violation_results = graph.query(violation_query)
for row in violation_results:
    print(f"  Tổng vi phạm: {row['ViolationCount']}")
    print(f"  Vi phạm có mức phạt: {row['WithPenalty']}")

print("\n=== Thống kê theo Mode (từ relationships) ===")
mode_query = """
MATCH ()-[r]->()
WHERE r.mode IS NOT NULL AND r.mode <> ''
RETURN r.mode as Mode, count(r) as Count
ORDER BY Count DESC
LIMIT 10
"""
mode_results = graph.query(mode_query)
for row in mode_results:
    print(f"  {row['Mode']}: {row['Count']} relationships")


In [ ]:
# Phân tích chi tiết các entities quan trọng
print("=== Top 15 Vi phạm phổ biến nhất ===")
violation_query = """
MATCH (v:VIOLATION)
OPTIONAL MATCH (v)-[r]-()
WITH v, count(DISTINCT r) as rel_count
RETURN v.name as Violation, v.description as Description, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 15
"""
violations = graph.query(violation_query)
for i, row in enumerate(violations, 1):
    desc = row.get('Description', '')[:60]
    print(f"{i:2d}. {row['Violation']} - {row['Connections']} kết nối")
    if desc:
        print(f"     {desc}")

print("\n=== Top 10 Phương tiện được đề cập ===")
vehicle_query = """
MATCH (v:VEHICLE)
OPTIONAL MATCH (v)-[r]-()
WITH v, count(DISTINCT r) as rel_count
RETURN v.name as Vehicle, v.description as Description, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 10
"""
vehicles = graph.query(vehicle_query)
for i, row in enumerate(vehicles, 1):
    desc = row.get('Description', '')[:40]
    print(f"{i:2d}. {row['Vehicle']} - {row['Connections']} kết nối")

print("\n=== Top 10 Mức phạt ===")
penalty_query = """
MATCH (p:PENALTY)
OPTIONAL MATCH (v)-[r:BỊ_PHẠT]->(p)
WITH p, count(v) as violation_count
RETURN p.name as Penalty, p.description as Description, violation_count as Violations
ORDER BY violation_count DESC
LIMIT 10
"""
penalties = graph.query(penalty_query)
for i, row in enumerate(penalties, 1):
    desc = row.get('Description', '')[:50]
    vio_count = row.get('Violations', 0)
    print(f"{i:2d}. {row['Penalty']}")
    print(f"     Vi phạm liên quan: {vio_count}")

print("\n=== Các quy định quan trọng ===")
regulation_query = """
MATCH (r:REGULATION)
OPTIONAL MATCH (r)-[rel]-()
WITH r, count(DISTINCT rel) as rel_count
WHERE rel_count > 0
RETURN r.name as Regulation, r.description as Description, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 10
"""
regulations = graph.query(regulation_query)
for i, row in enumerate(regulations, 1):
    desc = row.get('Description', '')[:50]
    print(f"{i:2d}. {row['Regulation']} - {row['Connections']} kết nối")

print("\n=== Phân tích Relationships theo Article ===")
article_query = """
MATCH ()-[r]->()
WHERE r.article IS NOT NULL
WITH r.article as Article, r.law_code as LawCode, count(r) as RelCount
RETURN Article, LawCode, RelCount
ORDER BY RelCount DESC
LIMIT 10
"""
article_results = graph.query(article_query)
for i, row in enumerate(article_results, 1):
    print(f"{i:2d}. Điều {row['Article']} ({row['LawCode']}): {row['RelCount']} relationships")

print("\n=== Phân tích Relationships theo Chapter ===")
chapter_query = """
MATCH ()-[r]->()
WHERE r.chapter IS NOT NULL AND r.chapter <> ''
WITH r.chapter as Chapter, r.chapter_title as Title, count(r) as RelCount
RETURN Chapter, Title, RelCount
ORDER BY RelCount DESC
LIMIT 10
"""
chapter_results = graph.query(chapter_query)
for i, row in enumerate(chapter_results, 1):
    title = row.get('Title', '')[:40]
    print(f"{i:2d}. {row['Chapter']}: {row['RelCount']} relationships - {title}")
